<a href="https://colab.research.google.com/github/alvinicdf-sketch/2026/blob/main/ProbationCalander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ics pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
# @title
import pandas as pd
from ics import Calendar, Event
from datetime import datetime
from google.colab import files

file_name = "★試用期人員名冊104-115(公式).xlsx"
sheet = "114-115年新人"

try:
    # 1. 讀取 Excel (跳過前兩行無關資訊，直接從標題列開始讀取)
    # 根據您的檔案內容，第三行 (index 2) 是真正的標題列
    df = pd.read_excel(file_name, sheet_name=sheet, header=2)

    # 清理欄位名稱的空格與換行
    df.columns = df.columns.astype(str).str.replace('\n', '').str.strip()

    c = Calendar()
    count = 0

    # 2. 開始逐行掃描
    for index, row in df.iterrows():
        # 強制指定欄位位置 (避免欄位名稱變動)
        # B=1(部門), C=2(單位), E=4(職稱), F=5(姓名), G=6(到職日), H=7(期滿日), J=9(考核通知日)
        dept = str(row.iloc[1]) if pd.notna(row.iloc[1]) else ""
        unit = str(row.iloc[2]) if pd.notna(row.iloc[2]) else ""
        job = str(row.iloc[4]) if pd.notna(row.iloc[4]) else ""
        name = str(row.iloc[5]).strip() if pd.notna(row.iloc[5]) else ""
        arrival_date_str = str(row.iloc[6]).split(' ')[0] if pd.notna(row.iloc[6]) else ""

        # 排除空行
        if not name or name == "nan" or name == "None":
            continue

        # 組合單位名稱 (例如：駐外 - 駐帛琉技術團)
        full_unit = f"{dept} - {unit}" if dept else unit

        # --- 類別 A：【考核通知】 (日期看 J 欄) ---
        notify_date = row.iloc[9]
        if pd.notna(notify_date):
            try:
                d_notify = pd.to_datetime(notify_date)
                e1 = Event()
                e1.name = f"【考核通知】{full_unit} - {name} {job}(到職日：{arrival_date_str})"
                e1.begin = d_notify.date() # 設定為日期物件，即為全日行程
                e1.make_all_day()
                c.events.add(e1)
                count += 1
            except: pass

        # --- 類別 B：【試用期滿】 (日期看 H 欄) ---
        expire_date = row.iloc[7]
        if pd.notna(expire_date):
            try:
                d_expire = pd.to_datetime(expire_date)
                e2 = Event()
                expire_str = d_expire.strftime('%Y/%m/%d')
                e2.name = f"【試用期滿】{full_unit} - {name} {job}(試用期滿：{expire_str})"
                e2.begin = d_expire.date() # 設定為日期物件，即為全日行程
                e2.make_all_day()
                c.events.add(e2)
                count += 1
            except: pass

    # 3. 匯出檔案
    if count > 0:
        output_name = "Outlook_AllDay_Events.ics"
        with open(output_name, 'w', encoding='utf-8-sig') as f:
            f.writelines(c.serialize_iter())
        print(f"✅ 處理完成！共生成 {count} 筆全日會議通知。")
        print(f"確認包含：劉育誠、林哲正 等人員資料。")
        files.download(output_name)
    else:
        print("❌ 錯誤：無法從指定欄位抓取到任何日期，請檢查 Excel 格式。")

except Exception as e:
    print(f"❌ 執行時出錯: {e}")

ModuleNotFoundError: No module named 'ics'

In [ ]:
# @title
import pandas as pd
from ics import Calendar, Event
from datetime import datetime, timedelta
from google.colab import files
import io

# --- 功能函數：遇假日(六、日)自動往前推至週五 ---
def adjust_to_friday(dt):
    if pd.isna(dt): return dt
    d = pd.to_datetime(dt)
    if d.weekday() == 5: # 週六
        return d - timedelta(days=1)
    elif d.weekday() == 6: # 週日
        return d - timedelta(days=2)
    return d

print("請點選下方『選擇檔案』按鈕，上傳您的 Excel 檔：")
uploaded = files.upload()

if uploaded:
    # 取得上傳的檔名
    file_name = list(uploaded.keys())[0]
    sheet_name = "114-115年新人"

    try:
        # 1. 讀取 Excel (從第 3 行開始為標題列，header=2)
        df = pd.read_excel(io.BytesIO(uploaded[file_name]), sheet_name=sheet_name, header=2)

        # 清理標題欄位
        df.columns = df.columns.astype(str).str.replace('\n', '').str.strip()

        c = Calendar()
        count = 0

        # 2. 逐行掃描資料
        for index, row in df.iterrows():
            # 欄位：F=5(姓名), G=6(到職日), H=7(期滿日), J=9(考核通知日)
            dept = str(row.iloc[1]) if pd.notna(row.iloc[1]) else ""
            unit = str(row.iloc[2]) if pd.notna(row.iloc[2]) else ""
            job = str(row.iloc[4]) if pd.notna(row.iloc[4]) else ""
            name = str(row.iloc[5]).strip() if pd.notna(row.iloc[5]) else ""
            arrival_date_str = str(row.iloc[6]).split(' ')[0] if pd.notna(row.iloc[6]) else ""

            # 過濾無名資料
            if not name or name == "nan" or name == "None" or name == "":
                continue

            full_unit = f"{dept} - {unit}" if dept else unit

            # --- 處理【考核通知】 (J 欄) ---
            raw_notify = row.iloc[9]
            if pd.notna(raw_notify):
                final_notify = adjust_to_friday(raw_notify)
                e1 = Event()
                e1.name = f"【考核通知】{full_unit} - {name} {job}(到職：{arrival_date_str})"
                e1.begin = final_notify.date()
                e1.make_all_day() # 設定為全日行程
                c.events.add(e1)
                count += 1

            # --- 處理【試用期滿】 (H 欄) ---
            raw_expire = row.iloc[7]
            if pd.notna(raw_expire):
                final_expire = adjust_to_friday(raw_expire)
                e2 = Event()
                expire_label = final_expire.strftime('%Y/%m/%d')
                e2.name = f"【試用期滿】{full_unit} - {name} {job}(期滿：{expire_label})"
                e2.begin = final_expire.date()
                e2.make_all_day() # 設定為全日行程
                c.events.add(e2)
                count += 1

        # 3. 匯出檔案
        if count > 0:
            output_file = "Outlook_AllDay_Fixed.ics"
            with open(output_file, 'w', encoding='utf-8-sig') as f:
                f.writelines(c.serialize_iter())
            print(f"\n✅ 處理完成！共生成 {count} 筆行程。")
            print("💡 說明：遇到週六、週日的通知已自動提前至週五。")
            files.download(output_file)
        else:
            print("\n❌ 讀取成功但沒抓到資料，請確認工作表名稱是否為 '114-115年新人'")

    except Exception as e:
        print(f"\n❌ 程式執行出錯: {e}")
else:
    print("未選擇任何檔案。")

請點選下方『選擇檔案』按鈕，上傳您的 Excel 檔：
